In [5]:
import pandas as pd

infra_upgrades = pd.read_excel("MTEP24 Appendix A Tranche 2.1 Projects.xlsx", sheet_name="Facilities")

# first row should be header 

infra_upgrades.columns = infra_upgrades.iloc[0]
infra_upgrades = infra_upgrades.iloc[1:].reset_index(drop=True)


infra_upgrades.head()

,Target Appendix,App ABC,Planning Region,Geographic Location by TO Member System,PrjID,Facility ID,Facility Type,Expected ISD,From Sub,To Sub,...,Share Status,Other Type,Transmission Project Type,Previous Quarter Expected ISD,Previous Quarter Planning Status,Previous Quarter Estimated Cost,Facility Cost Change %,Cost Est. Variance,In Service Date Change (Months),Fac. Delay Reason
0,A in MTEP24,A,West,"GRE, MP, MRES, OTP, XEL",19,NaN,LN,2032-06-01 00:00:00,MN/ND State Line,Bison,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A in MTEP24,A,West,XEL,19,NaN,Sub,2032-06-01 00:00:00,Bison,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A in MTEP24,A,West,XEL,19,NaN,Sub,2032-06-01 00:00:00,Cass County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A in MTEP24,A,West,XEL,19,NaN,LN,2032-06-01 00:00:00,Cass County,Red River,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A in MTEP24,A,West,XEL,19,NaN,Sub,2032-06-01 00:00:00,Red River,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# include a mapping from state abbreviations to state names
state_abbreviations = {
    'TX': 'Texas',
    'CA': 'California',
    'AZ': 'Arizona',
    'NM': 'New Mexico',
    'OK': 'Oklahoma',
    'KS': 'Kansas',
    'NE': 'Nebraska',
    'MO': 'Missouri',
    'IA': 'Iowa',
    'MN': 'Minnesota',
    'WI': 'Wisconsin',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'OH': 'Ohio',
    'MI': 'Michigan',
    'PA': 'Pennsylvania',
    'NY': 'New York',
    'VA': 'Virginia',
    'NC': 'North Carolina',
    'SC': 'South Carolina',
    'GA': 'Georgia',
    'AL': 'Alabama',
    'TN': 'Tennessee',
    'AR': 'Arkansas',
    'LA': 'Louisiana',
    'MS': 'Mississippi',
    'KY': 'Kentucky',
    'WV': 'West Virginia',
    'MD': 'Maryland',
    'DE': 'Delaware',
    'NJ': 'New Jersey',
    'CT': 'Connecticut',
    'RI': 'Rhode Island',
    'MA': 'Massachusetts',
    'ME': 'Maine',
    'NH': 'New Hampshire',
    'VT': 'Vermont',
    'ID': 'Idaho',
    'WY': 'Wyoming',
    'MT': 'Montana',
    'ND': 'North Dakota',
    'SD': 'South Dakota',
    'NE': 'Nebraska',
    'IA': 'Iowa',
    'MO': 'Missouri',
    'AR': 'Arkansas',
    'OK': 'Oklahoma',
    'TX': 'Texas',
    'NM': 'New Mexico',
    'AZ': 'Arizona',
    'CA': 'California',
    'NV': 'Nevada', 
    'WA': 'Washington',
    'OR': 'Oregon',
    'UT': 'Utah',
    'ID': 'Idaho',
    'WY': 'Wyoming',
    'MT': 'Montana',
    'ND': 'North Dakota',
    'SD': 'South Dakota',
    'NE': 'Nebraska',
    'IA': 'Iowa',
    'MO': 'Missouri',
    'AR': 'Arkansas',
        }
print(len(set(state_abbreviations.keys())))

46


In [15]:
import requests
import pandas as pd
from collections import defaultdict
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def get_substations_for_state(state_name):
    """
    Fetch ALL named substations in a given state (admin_level=4).
    Returns a dict mapping substation_name -> list of (lat, lon).
    (We use a list because sometimes multiple OSM features may share the same name.)
    """
    logger.info(f"Fetching substations for state: {state_name}")
    overpass_url = "https://overpass-api.de/api/interpreter"
    
    query = f"""
    [out:json];
    area["name"="{state_name}"]["admin_level"="4"]->.searchArea;
    (
      node["power"="substation"]["name"](area.searchArea);
      way["power"="substation"]["name"](area.searchArea);
      relation["power"="substation"]["name"](area.searchArea);
    );
    out center;
    """
    
    response = requests.post(overpass_url, data={'data': query})
    response.raise_for_status()
    data = response.json()
    
    # Build a dictionary: name -> list of coordinates
    substation_dict = defaultdict(list)
    for el in data.get('elements', []):
        # name
        name = el.get('tags', {}).get('name', None)
        if not name:
            continue
        # coordinates
        if el['type'] == 'node':
            lat, lon = el['lat'], el['lon']
        else:
            # ways/relations => use 'center'
            center = el.get('center', {})
            lat, lon = center.get('lat'), center.get('lon')
        if lat is not None and lon is not None:
            substation_dict[name].append((lat, lon))
            logger.debug(f"Found substation: {name} at coordinates ({lat}, {lon})")
    
    logger.info(f"Found {len(substation_dict)} substations in {state_name}")
    return dict(substation_dict)

# Example usage with your dataframe 'infra_upgrades':
def populate_substation_coords(df):
    # Suppose df has columns: 'From Sub', 'To Sub', 'State'
    # And you want to append columns for from/to coords
    
    logger.info("Starting substation coordinate population")
    
    # Get unique states
    unique_states = df['State'].unique()
    logger.info(f"Processing {len(unique_states)} unique states")
    
    # Create a dictionary of state->(substation_name->coords_list)
    state_substations_cache = {}
    
    for st in unique_states:
        try:
            # Download data once per state
            state_name = state_abbreviations[st]
            logger.info(f"Processing state: {st} ({state_name})")
            station_lookup = get_substations_for_state(state_name)
            state_substations_cache[st] = station_lookup
            logger.info(f"Fetched {len(station_lookup)} unique substation names for state={st}")
        except Exception as e:
            logger.error(f"Error fetching data for state={st}: {e}")
            state_substations_cache[st] = {}
    
    # For each row, do a local lookup
    from_coords = []
    to_coords = []
    
    logger.info("Processing individual substations")
    for idx, row in df.iterrows():
        logger.debug(f"Processing row {idx}")
        # Expand the substation name if needed
        from_substation_name = f"{row['From Sub']} Substation"
        to_substation_name = f"{row['To Sub']} Substation" if pd.notna(row['To Sub']) else ""
        st = row['State']
        
        # default coordinate
        found_from = None
        found_to = None
        
        # Check if we have a dictionary for that state
        station_lookup = state_substations_cache.get(st, {})
        
        # Attempt exact matching or partial matching
        if from_substation_name in station_lookup:
            # If multiple coords found (multiple OSM features share the same name),
            # you can decide how to handle – e.g., pick the first
            found_from = station_lookup[from_substation_name][0]
            logger.debug(f"Found coordinates for {from_substation_name}: {found_from}")
        else:
            logger.debug(f"No coordinates found for {from_substation_name}")
        
        if to_substation_name in station_lookup:
            found_to = station_lookup[to_substation_name][0]
            logger.debug(f"Found coordinates for {to_substation_name}: {found_to}")
        elif to_substation_name:
            logger.debug(f"No coordinates found for {to_substation_name}")
        
        from_coords.append(found_from)
        to_coords.append(found_to)
    
    # 4) Attach to dataframe
    logger.info("Attaching coordinates to dataframe")
    df['From Sub Coordinates'] = from_coords
    df['To Sub Coordinates'] = to_coords

    logger.info("Finished populating substation coordinates")
    return df

# Then call it on your infra_upgrades dataframe
infra_upgrades = populate_substation_coords(infra_upgrades)


2025-03-08 15:15:11,350 - INFO - Starting substation coordinate population
2025-03-08 15:15:11,400 - INFO - Processing 11 unique states
2025-03-08 15:15:11,401 - INFO - Processing state: ND (North Dakota)
2025-03-08 15:15:11,402 - INFO - Fetching substations for state: North Dakota
2025-03-08 15:15:41,384 - INFO - Found 134 substations in North Dakota
2025-03-08 15:15:41,397 - INFO - Fetched 134 unique substation names for state=ND
2025-03-08 15:15:41,398 - INFO - Processing state: MN (Minnesota)
2025-03-08 15:15:41,398 - INFO - Fetching substations for state: Minnesota
2025-03-08 15:16:14,050 - INFO - Found 525 substations in Minnesota
2025-03-08 15:16:14,055 - INFO - Fetched 525 unique substation names for state=MN
2025-03-08 15:16:14,056 - INFO - Processing state: WI (Wisconsin)
2025-03-08 15:16:14,056 - INFO - Fetching substations for state: Wisconsin
2025-03-08 15:16:46,812 - INFO - Found 556 substations in Wisconsin
2025-03-08 15:16:46,816 - INFO - Fetched 556 unique substation n

In [19]:
infra_upgrades

,Target Appendix,App ABC,Planning Region,Geographic Location by TO Member System,PrjID,Facility ID,Facility Type,Expected ISD,From Sub,To Sub,...,Transmission Project Type,Previous Quarter Expected ISD,Previous Quarter Planning Status,Previous Quarter Estimated Cost,Facility Cost Change %,Cost Est. Variance,In Service Date Change (Months),Fac. Delay Reason,From Sub Coordinates,To Sub Coordinates
0,A in MTEP24,A,West,"GRE, MP, MRES, OTP, XEL",19,NaN,LN,2032-06-01 00:00:00,MN/ND State Line,Bison,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"(46.9507742, -97.0758717)"
1,A in MTEP24,A,West,XEL,19,NaN,Sub,2032-06-01 00:00:00,Bison,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(46.9507742, -97.0758717)",None
2,A in MTEP24,A,West,XEL,19,NaN,Sub,2032-06-01 00:00:00,Cass County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(46.875282, -96.8498435)",None
3,A in MTEP24,A,West,XEL,19,NaN,LN,2032-06-01 00:00:00,Cass County,Red River,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(46.875282, -96.8498435)","(46.8804615, -96.808145)"
4,A in MTEP24,A,West,XEL,19,NaN,Sub,2032-06-01 00:00:00,Red River,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(46.8804615, -96.808145)",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,A in MTEP24,A,Central,"AEP (PJM), ComEd (PJM)",42,NaN,LN,2032-06-01 00:00:00,Babcock,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(41.5339647, -87.1022387)",None
318,A in MTEP24,A,Central,NIPSCO,42,NaN,Sub,2032-06-01 00:00:00,Burr Oak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(41.2576945, -86.4225728)",None
319,A in MTEP24,A,Central,NIPSCO,42,NaN,LN,2032-06-01 00:00:00,Burr Oak,Schahfer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(41.2576945, -86.4225728)",None
320,A in MTEP24,A,Central,NIPSCO,42,NaN,LN,2032-06-01 00:00:00,Burr Oak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(41.2576945, -86.4225728)",None


In [17]:
# save file as csv
infra_upgrades.to_csv('infra_upgrades_coordinates.csv', index=False)